In [1]:
import pandas as pd
import numpy as np
import re
#importar unicode
from unidecode import unidecode

In [2]:
TOKEN_SPLIT_RE = re.compile(r'^(?P<cod>[A-Za-z]{1,4})\s+(?P<rest>.+)$')

In [3]:
dfClass =pd.read_excel('data/Estudo_de_Garantias_v3.xlsx', sheet_name='Classificação', header =1)
df_original = pd.read_csv('data/df_tidy_simp.csv')
df = df_original.copy()

df.head(5)

,Fundo,%PL,Norm.,Ativo,Garantia,Nota
0,VGIR11,0.053395,0.066457,22L1013767,AF SPE,2.0
1,VGIR11,0.051467,0.064057,22E1211649,AF Terreno e SPE + CF,3.0
2,VGIR11,0.049767,0.061942,23K1511855,AF SPE + Fiança,2.0
3,VGIR11,0.046417,0.057773,24A1828538,AF Imóvel + Aval,3.0
4,VGIR11,0.042653,0.053088,22K1684666,AF SPE,2.0


In [4]:
# df com 
df = df[['Fundo' , 'Ativo', 'Garantia']]

In [5]:
regex = r'\s*(?:\+|-|,|;|\bou\b|\be\b|\bem\b|\bde\b|\bda\b|\bdos\b|\bdo\b|\be/?ou\b|\(\w+\)|•)\s*'
df['Garantia'] = df['Garantia'].str.replace(r'^\s*(?:-+|•+|GARANTIAS)\s*', '', regex=True)


In [6]:
def limpar_celula(x):
    if not isinstance(x, str):
        return x
    x = x.strip(" ;.•")                    # remove pontuação
    x = re.sub(r'^\d+\s*', '', x)          # remove número no início
    x = re.sub(r'\s*\d+$', '', x)          # remove número no fim
    return x.strip()

In [7]:
df_split = df['Garantia'].str.split(regex, expand=True)

# Aplica limpeza por célula (pontuação + números no início/fim)
df_split = df_split.applymap(limpar_celula)

# Nomeia as colunas
df_split.columns = [f'Garantia_{i+1}' for i in range(df_split.shape[1])]

# Junta com as colunas originais
df_split = pd.concat([df[['Fundo', 'Ativo']], df_split], axis=1)
df_split.iloc[1:10].dropna(axis=1, how='all')

/var/folders/st/r72pjbts3bb1zfh2s1r4pzw00000gn/T/ipykernel_26613/4078454562.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(limpar_celula)


,Fundo,Ativo,Garantia_1,Garantia_2,Garantia_3,Garantia_4
1,VGIR11,22E1211649,AF Terreno,SPE,CF,None
2,VGIR11,23K1511855,AF SPE,Fiança,None,None
3,VGIR11,24A1828538,AF Imóvel,Aval,None,None
4,VGIR11,22K1684666,AF SPE,None,None,None
5,VGIR11,22D0836679,clean,None,None,None
6,VGIR11,24G1972260,AF Imóvel,CF,Coobrigação,Aval
7,VGIR11,24E1753141,AF Imóvel,Terreno,CF,Aval
8,VGIR11,23J1928151,AF Imóvel,SPE,CF,Fiança
9,VGIR11,21L0002653,AF Terreno,None,None,None


In [8]:
tipos_garantia = set(dfClass['Tipos de Garantia'].dropna().str.strip().unique())
codigo = set(dfClass['Código'].dropna().str.strip().unique())
subclasses = set(dfClass['Subclasse'].dropna().str.strip().unique())


In [9]:
def normalizar(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = unidecode(s).lower()
    s = re.sub(r'\s+', ' ', s).strip()
    return s


In [10]:
tipos_norm = {normalizar(t) for t in tipos_garantia}
codigos = {c.upper() for c in codigo}
subs_norm = {normalizar(s) for s in subclasses}
prefix_tipo = {t.split(' ', 1)[0]: t for t in tipos_norm}

In [11]:
alias = {
    # ─── Subclasses ──────────────────────────────────────────
    'imoveis'      : 'imovel',
    'imóvel'       : 'imovel',          # caso apareça com acento
    'imóveis'      : 'imovel',
    'alugueis'     : 'aluguéis',
    'graos'        : 'grãos',
    'grao'        : 'grãos',
    'aluguel'     : 'aluguéis',
    'terrenoa'     : 'terreno',
    'cotas'        : 'cotas de fundo (fip, fii etc.) e ações',
    'quotas'       : 'cotas de fundo (fip, fii etc.) e ações',
    'recebiveis'   : 'recebíveis/ crédito/ direito creditório',
    'sobrecolateral':'sobrecolateral',
    'sócios pessoa física': 'sócios',
    'sócios pessoa jurídica': 'sócios',
    'sócios pf': 'sócios',
    'spes'       : 'spe',
    'terrenos'   : 'terreno',
    'graos'      : 'grãos',
    'grãos'      : 'grãos',

    # ─── Tipos de Garantia ───────────────────────────────────
    'cash'         : 'cash sweep',
    'fianca'       : 'fiança',
    'fiança'       : 'fianca',
    
}


In [12]:
def keep_token(token):
    """Converte token bruto em lista [ tipo | subclasse | código ]
       ou [] se deve ser descartado."""

    # 1) se vier lista, trata item a item (recursivo)
    if isinstance(token, list):
        out = []
        for item in token:
            out.extend(keep_token(item))
        return out

    # 2) se não é string útil, descarta
    if not isinstance(token, str) or token.strip() == "":
        return []

    # 3) limpeza básica
    t = normalizar(token)
    t = re.sub(r'[\d$\.]+', '', t)      # remove números, cifrões, ponto
    t = t.strip(',;() ')                # remove pontuação solta
    if t == '':
        return []

    # 4) descarta percentuais ou “reserva” irrelevante
    if '%' in t:
        return []
    if t.startswith('reserva') and 'fundo' not in t:
        return []

    # 5) aplica alias (sempre para nomes existentes)
    t = alias.get(t, t)
    t = normalizar(t)  # re-normaliza caso o alias tenha acento

    out = []

    # 6) padrão “COD – resto”
    m = TOKEN_SPLIT_RE.match(t)
    if m:
        cod = m.group('cod').upper()
        rest = normalizar(m.group('rest'))

        # 6-A) tenta casar o resto completo antes de quebrar
        if rest in subs_norm:
            out.append(rest)
        else:
            for piece in rest.split():
                if piece in subs_norm:
                    out.append(piece)

        # 6-B) código sempre vai pro output se for reconhecido
        if cod in codigos:
            out.append(cod)

        return list(dict.fromkeys(out))  # remove duplicatas mantendo ordem

    # 7) prefixo que indica tipo
    first = t.split(' ', 1)[0]
    if first in prefix_tipo:
        return [prefix_tipo[first]]

    # 8) token é um tipo ou subclasse diretamente
    if t in tipos_norm or t in subs_norm:
        return [t]

    # 9) token é um código puro
    if token.upper() in codigos:
        return [token.upper()]

    # 10) ruído → nada
    return []


In [13]:
def filtra_linha(row):
    kept = []
    for cell in row.to_numpy():        # garante só valores
        kept.extend(keep_token(cell))
    return pd.Series(kept)


In [14]:
# 1) pega apenas as colunas Garantia_*
gar_cols = df_split.filter(like='Garantia_')

# 2) aplica a limpeza linha‑a‑linha
tmp = gar_cols.apply(filtra_linha, axis=1)

# 3) renomeia as colunas 0,1,2… ⇒ G1,G2,G3…
tmp.columns = [f'G{i+1}' for i in range(tmp.shape[1])]

# 4) junta Fundo e Ativo de volta
df_clean = pd.concat([df_split[['Fundo', 'Ativo']], tmp], axis=1)


In [15]:
ruido = (df_split.filter(like='Garantia_')
                   .stack()
                   .dropna()
                   .apply(lambda x: keep_token(x)==[]))
print(f"Ruído remanescente: {ruido.mean():.2%}")


Ruído remanescente: 47.16%


In [16]:

ruido = (gar_cols.stack()
                   .dropna()
                   .loc[lambda s: s.apply(lambda x: keep_token(x) == [])])


In [17]:
df_clean = pd.concat([df_split[['Fundo', 'Ativo']], tmp], axis=1)

In [18]:
df_clean.to_csv('data/garantias_limpas.csv', index=False)     # arquivo pronto
df_clean.to_excel('data/garantias_limpas.xlsx', index=False)
df_clean.head(50)

,Fundo,Ativo,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12
0,VGIR11,22L1013767,spe,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VGIR11,22E1211649,terreno,AF,spe,CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VGIR11,23K1511855,spe,AF,fianca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VGIR11,24A1828538,imovel,AF,aval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VGIR11,22K1684666,spe,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VGIR11,22D0836679,clean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,VGIR11,24G1972260,imovel,AF,CF,aval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VGIR11,24E1753141,imovel,AF,terreno,CF,aval,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,VGIR11,23J1928151,imovel,AF,spe,CF,fianca,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,VGIR11,21L0002653,terreno,AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
top30 = ruido.value_counts().head(60)
print(top30)


                                                                  808
Promessa                                                           80
%                                                                  58
Obras                                                              56
Coobrigação                                                        52
Reserva                                                            48
coobrigação                                                        44
Reserva (02 PMTs)                                                  37
Obra                                                               35
cedente                                                            34
Reserva (03 PMTs)                                                  34
sócios pessoa física                                               31
jurídica                                                           27
direitos creditórios                                               26
vendas futuras      

In [20]:
# df_clean já contém G1, G2, …;  conjuntos válidos = tipos_norm, codigos, subs_norm
gar_cols = df_clean.filter(like='G')

tokens_suspeitos = (gar_cols.stack(dropna=True)
                               .unique()
                               .tolist())

# Remove tudo que REALMENTE é válido
tokens_suspeitos = [
    tok for tok in tokens_suspeitos
    if (tok.upper() not in codigos and
        normalizar(tok) not in tipos_norm and
        normalizar(tok) not in subs_norm)
]

print(f"{len(tokens_suspeitos)} tokens ainda fora da classificação:")
print(sorted(tokens_suspeitos)[:50])        # mostra só os 50 primeiros


0 tokens ainda fora da classificação:
[]


/var/folders/st/r72pjbts3bb1zfh2s1r4pzw00000gn/T/ipykernel_26613/4022207843.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  tokens_suspeitos = (gar_cols.stack(dropna=True)
